In [4]:
# !python -m pip install tensorflow[and-cuda]

!pip install transformers



In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# from transformers import BertTokenizer, TFBertModel


# Load the data
# train = pd.read_csv('/Users/dkamboj6/Recitation/ECE-461P-Term-Project/mercari-price-suggestion-challenge/train.tsv',  delimiter='\t')

# test = pd.read_csv('/Users/dkamboj6/Recitation/ECE-461P-Term-Project/mercari-price-suggestion-challenge/test.tsv', delimiter='\t')

train= pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/mercari-price-suggestion-challenge/train.tsv/train.tsv',delimiter='\t')
test=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/mercari-price-suggestion-challenge/test.tsv/test.tsv',delimiter='\t')

In [9]:
# Fill missing values

train.fillna('No Data', inplace=True)
test.fillna('No Data', inplace=True)
